## Грузим библиотеки

In [ ]:
%pip install transformers
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 451 kB 1.6 MB/s            
     |████████████████████████████████| 78 kB 3.9 MB/s            
     |████████████████████████████████| 36.0 MB 6.5 MB/s            
     |████████████████████████████████| 132 kB 8.1 MB/s            
     |████████████████████████████████| 110 kB 8.8 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently tak

## Датасет

In [ ]:
#!g1.1
from datasets import load_dataset
dataset = load_dataset('blinoff/kinopoisk')

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

  Dataset kinopoisk downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/blinoff___kinopoisk/simple/1.0.0/62f52027aea59f64f49c7b16165b82cb4dc45031bad3660c2719bf2a6ea4a44e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#!g1.1
import pandas as pd

def prepare_texts(texts):
    res = []
    for t in texts:
        res.append(t.strip().replace('\n', ' '))
    return res

df = pd.DataFrame({'text': prepare_texts(dataset['train']['content']), 'grade': dataset['train']['grade3']})

In [ ]:
#!g1.1
df.head()

,text,grade
0,"""Блеф» — одна из моих самых любимых комедий. ...",Good
1,Адриано Челентано продолжает радовать нас свои...,Good
2,"Несомненно, это один из великих фильмов 80-х г...",Good
3,Эта фраза на мой взгляд отражает сюжет несомне...,Good
4,"- как пела Земфира, скорее всего, по совершенн...",Neutral


In [ ]:
#!g1.1
df['grade'].value_counts()

Good       27264
Bad         4751
Neutral     4576
Name: grade, dtype: int64

In [ ]:
#!g1.1
id2label = ['Good', 'Neutral', 'Bad']
labels2id = {label: i for i, label in enumerate(id2label)}

df['label'] = df['grade'].apply(lambda x: labels2id[x])

In [ ]:
#!g1.1
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df['text'].to_numpy(), df['label'].to_numpy(), test_size=0.2, random_state=42, shuffle=True, stratify=df['label'].to_numpy())
x_str, x_nstr, y_str, y_nstr = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

In [ ]:
#!g1.1
import numpy as np

def get_data_not_stratifyed(x, y):
    y_tmp_0 = y[y == 0]
    x_tmp_0 = x[y == 0]
    y_tmp_n_0 = y[y != 0]
    x_tmp_n_0 = x[y != 0]
    length_0 = len(y_tmp_0) // 20
    return np.concatenate((x_tmp_0[:length_0], x_tmp_n_0)), np.concatenate((y_tmp_0[:length_0], y_tmp_n_0))


x_nstr, y_nstr = get_data_not_stratifyed(x_nstr, y_nstr)

print('Count of 0 in distribution:\t', len(y_str[y_str == 0]), '\t count of 0 out of distribution:\t', len(y_nstr[y_nstr == 0]))
print('Count of 1 in distribution:\t', len(y_str[y_str == 1]), '\t count of 1 out of distribution:\t', len(y_nstr[y_nstr == 1]))
print('Count of 2 in distribution:\t', len(y_str[y_str == 2]), '\t count of 2 out of distribution:\t', len(y_nstr[y_nstr == 2]))

Count of 0 in distribution:	 2727 	 count of 0 out of distribution:	 136
Count of 1 in distribution:	 457 	 count of 1 out of distribution:	 458
Count of 2 in distribution:	 475 	 count of 2 out of distribution:	 475


## Код для обучения

In [ ]:
#!g1.1
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#!g1.1
from torch.utils.data import Dataset, DataLoader

BATCH_SIZE = 32

class KinopoiskDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device=torch.device('cpu')):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = labels
        self.device = device

    def to(self, device):
        self.device = device
        return self

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, return_tensors='pt')
        return {k: v.to(self.device).flatten() for k, v in tokens.items()}, torch.tensor(self.labels[idx]).to(self.device)

train_dataset = KinopoiskDataset(x_train.tolist(), y_train.tolist(), tokenizer, device=device)
str_dataset = KinopoiskDataset(x_str.tolist(), y_str.tolist(), tokenizer, device=device)
nstr_dataset = KinopoiskDataset(x_nstr.tolist(), y_nstr.tolist(), tokenizer, device=device)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
str_loader = DataLoader(str_dataset, batch_size=BATCH_SIZE)
nstr_loader = DataLoader(nstr_dataset, batch_size=BATCH_SIZE)

In [ ]:
#!g1.1
import torch.functional as F
from sklearn.metrics import accuracy_score
from tqdm import tqdm

criterion = torch.nn.CrossEntropyLoss()

def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for x, y in tqdm(train_loader):
        outputs = model(**x)
        loss = criterion(outputs.logits, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    return total_loss / len(train_loader)


@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    total_loss = 0
    y_true = []
    y_pred = []
    for x, y in test_loader:
        outputs = model(**x)
        loss = criterion(outputs.logits, y)
        total_loss += loss.item()
        y_true.extend(y.tolist())
        y_pred.extend(torch.argmax(outputs.logits, dim=1).tolist())

    return total_loss / len(test_loader), accuracy_score(y_true, y_pred)


def train_epochs(model, train_loader, test_loader_str, test_loader_nstr, optimizer, n_epochs):
    for epoch in range(n_epochs):
        train_loss = train(model, train_loader, optimizer)
        print('Epoch', epoch)
        print('Train loss:\t', train_loss)
        test_loss, test_acc = evaluate(model, test_loader_str)
        print('Test loss ID:\t', test_loss, '\tTest acc ID:\t', test_acc)
        test_loss, test_acc = evaluate(model, test_loader_nstr)
        print('Test loss OOD:\t', test_loss, '\tTest acc OOD:\t', test_acc)

In [ ]:
#!g1.1
def freeze_embeddings(model):
    params = list(model.bert.embeddings.parameters())
    for i, param in enumerate(params):
        if i == len(params) - 1:
            return

        param.requires_grad = False


def unfreeze_embeddings(model):
    for param in model.bert.embeddings.parameters():
        param.requires_grad = True

## Обучение

## lr = 3e-4


In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
freeze_embeddings(model)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 10)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.597517860375467
Test loss ID:	 0.5154306884693063 	Test acc ID:	 0.8045914184203334
Test loss OOD:	 1.2572726519668804 	Test acc OOD:	 0.4667913938260056
Epoch 1
Train loss:	 0.5001001001544337
Test loss ID:	 0.5227222008549649 	Test acc ID:	 0.8108772888767423
Test loss OOD:	 1.417731615659945 	Test acc OOD:	 0.43405051449953225
Epoch 2
Train loss:	 0.4705772715025261
Test loss ID:	 0.506436587675758 	Test acc ID:	 0.80896419786827
Test loss OOD:	 1.350017574889695 	Test acc OOD:	 0.45650140318054255
Epoch 3
Train loss:	 0.4516633277138074
Test loss ID:	 0.4992341203534085 	Test acc ID:	 0.8108772888767423
Test loss OOD:	 1.2579573545166676 	Test acc OOD:	 0.47895229186155286
Epoch 4
Train loss:	 0.43514135798307063
Test loss ID:	 0.505113650534464 	Test acc ID:	 0.8070511068597978
Test loss OOD:	 1.28718791124137 	Test acc OOD:	 0.46024321796071094
Epoch 5
Train loss:	 0.42189645414632526
Test loss ID:	 0.4936322160389112 	Test acc ID:	 0.8136102760317027
Test 

In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 10)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.5376327929112429
Test loss ID:	 0.4556072084800057 	Test acc ID:	 0.8196228477726155
Test loss OOD:	 1.0780626081806772 	Test acc OOD:	 0.5266604303086997
Epoch 1
Train loss:	 0.422198479296894
Test loss ID:	 0.4983040225246678 	Test acc ID:	 0.8190762503416233
Test loss OOD:	 1.308023927075898 	Test acc OOD:	 0.5126286248830683
Epoch 2
Train loss:	 0.3539936543325257
Test loss ID:	 0.5681606997614321 	Test acc ID:	 0.816069964471167
Test loss OOD:	 1.5187118715223145 	Test acc OOD:	 0.5173058933582788
Epoch 3
Train loss:	 0.29062927213645046
Test loss ID:	 0.6325391137081644 	Test acc ID:	 0.8065045094288057
Test loss OOD:	 1.6023894910536267 	Test acc OOD:	 0.5126286248830683
Epoch 4
Train loss:	 0.24022004318835793
Test loss ID:	 0.6906029000878334 	Test acc ID:	 0.7947526646624761
Test loss OOD:	 1.5793677583775099 	Test acc OOD:	 0.5509822263797942
Epoch 5
Train loss:	 0.19640631518158758
Test loss ID:	 0.7379461337690768 	Test acc ID:	 0.7906531839300355
Te

In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
freeze_embeddings(model)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 5)

unfreeze_embeddings(model)
train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 5)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.5840054696212049
Test loss ID:	 0.5122990868661714 	Test acc ID:	 0.8051380158513255
Test loss OOD:	 1.290705388302312 	Test acc OOD:	 0.4106641721234799
Epoch 1
Train loss:	 0.4982157230214343
Test loss ID:	 0.48452109318712483 	Test acc ID:	 0.8127903798852145
Test loss OOD:	 1.2178758608506006 	Test acc OOD:	 0.43592142188961647
Epoch 2
Train loss:	 0.471726274164648
Test loss ID:	 0.479478483096413 	Test acc ID:	 0.8174364580486472
Test loss OOD:	 1.152577225557145 	Test acc OOD:	 0.4920486435921422
Epoch 3
Train loss:	 0.4515916393102844
Test loss ID:	 0.48232384609139484 	Test acc ID:	 0.8155233670401749
Test loss OOD:	 1.2647453596925033 	Test acc OOD:	 0.45088868101029
Epoch 4
Train loss:	 0.43412431943449165
Test loss ID:	 0.49058618765810263 	Test acc ID:	 0.8155233670401749
Test loss OOD:	 1.1468411567894852 	Test acc OOD:	 0.510757717492984
Epoch 0
Train loss:	 0.44992196863629125
Test loss ID:	 0.4973970712526985 	Test acc ID:	 0.8073244055752938
Tes

## lr = 1e-4

In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
freeze_embeddings(model)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 10)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.5649307834646089
Test loss ID:	 0.48659713747708694 	Test acc ID:	 0.80896419786827
Test loss OOD:	 1.1942725164048813 	Test acc OOD:	 0.44808231992516373
Epoch 1
Train loss:	 0.4785665628688583
Test loss ID:	 0.47001336579737457 	Test acc ID:	 0.8168898606176551
Test loss OOD:	 1.1656181273214958 	Test acc OOD:	 0.4855004677268475
Epoch 2
Train loss:	 0.4487028376977952
Test loss ID:	 0.4719176778326864 	Test acc ID:	 0.8152500683246788
Test loss OOD:	 1.1323596083504313 	Test acc OOD:	 0.5070159027128157
Epoch 3
Train loss:	 0.42345910031613104
Test loss ID:	 0.46858665087948675 	Test acc ID:	 0.8223558349275758
Test loss OOD:	 1.1329650116317413 	Test acc OOD:	 0.5116931711880262
Epoch 4
Train loss:	 0.40141297424914407
Test loss ID:	 0.4725770688575247 	Test acc ID:	 0.8198961464881115
Test loss OOD:	 1.0961273750838112 	Test acc OOD:	 0.5463049579045838
Epoch 5
Train loss:	 0.37803025704915405
Test loss ID:	 0.48766616200623303 	Test acc ID:	 0.8234490297895

In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 10)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.5346942067797719
Test loss ID:	 0.45297387825406116 	Test acc ID:	 0.8261820169445203
Test loss OOD:	 1.0947292451472843 	Test acc OOD:	 0.5051449953227315
Epoch 1
Train loss:	 0.41374693624471703
Test loss ID:	 0.4648855119943619 	Test acc ID:	 0.8289150040994807
Test loss OOD:	 1.208124396226862 	Test acc OOD:	 0.5070159027128157
Epoch 2
Train loss:	 0.35150564208708174
Test loss ID:	 0.5132586041870324 	Test acc ID:	 0.8204427439191035
Test loss OOD:	 1.3234887502210981 	Test acc OOD:	 0.5182413470533208
Epoch 3
Train loss:	 0.28897688191770854
Test loss ID:	 0.5567356593582942 	Test acc ID:	 0.8056846132823176
Test loss OOD:	 1.1806016325512354 	Test acc OOD:	 0.6024321796071095
Epoch 4
Train loss:	 0.23818839270967604
Test loss ID:	 0.647305758005899 	Test acc ID:	 0.8002186389723969
Test loss OOD:	 1.6546788645141266 	Test acc OOD:	 0.5144995322731525
Epoch 5
Train loss:	 0.17530353022026804
Test loss ID:	 0.7132118812073832 	Test acc ID:	 0.791746378792019

In [ ]:
#!g1.1
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
freeze_embeddings(model)

train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 5)

unfreeze_embeddings(model)
train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 5)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Epoch 0
Train loss:	 0.5827813289041728
Test loss ID:	 0.5069454217734544 	Test acc ID:	 0.7972123531019404
Test loss OOD:	 1.258787292329704 	Test acc OOD:	 0.41627689429373244
Epoch 1
Train loss:	 0.4971626570804523
Test loss ID:	 0.49512369179207344 	Test acc ID:	 0.8037715222738453
Test loss OOD:	 1.3088763226919315 	Test acc OOD:	 0.40037418147801684
Epoch 2
Train loss:	 0.4600279007941647
Test loss ID:	 0.47416845326838286 	Test acc ID:	 0.816616561902159
Test loss OOD:	 1.1752765612567173 	Test acc OOD:	 0.480823199251637
Epoch 3
Train loss:	 0.4327209027901373
Test loss ID:	 0.4808907555497211 	Test acc ID:	 0.8122437824542225
Test loss OOD:	 1.215663591728491 	Test acc OOD:	 0.47427502338634236
Epoch 4
Train loss:	 0.4042392864416206
Test loss ID:	 0.4884067331967146 	Test acc ID:	 0.8152500683246788
Test loss OOD:	 1.1503317121635466 	Test acc OOD:	 0.49859681945743684
Epoch 0
Train loss:	 0.3888397772536903
Test loss ID:	 0.4750152393527653 	Test acc ID:	 0.8179830554796392


In [ ]:
#!g1.1
train_epochs(model, train_loader, str_loader, nstr_loader, optimizer, 5)

100%|██████████| 915/915 [02:48<00:00,  5.44it/s]


Epoch 0
Train loss:	 0.14255614224719546
Test loss ID:	 0.7466898287444012 	Test acc ID:	 0.7944793659469801
Test loss OOD:	 1.9736767263316057 	Test acc OOD:	 0.5023386342376053
Epoch 1
Train loss:	 0.12120253589828728
Test loss ID:	 0.8213532206480918 	Test acc ID:	 0.8040448209893414
Test loss OOD:	 2.0777307733729042 	Test acc OOD:	 0.5304022450888681
Epoch 2
Train loss:	 0.08344256124367602
Test loss ID:	 0.8870184972234394 	Test acc ID:	 0.7952992620934681
Test loss OOD:	 2.003106558585868 	Test acc OOD:	 0.5603367633302152
Epoch 3
Train loss:	 0.07381773127619523
Test loss ID:	 0.8799423997816832 	Test acc ID:	 0.7813610276031703
Test loss OOD:	 2.1186257493408287 	Test acc OOD:	 0.5154349859681946
Epoch 4
Train loss:	 0.06188736435667169
Test loss ID:	 0.970144082751611 	Test acc ID:	 0.7862804044820989
Test loss OOD:	 2.032825766459984 	Test acc OOD:	 0.5743685687558466


# 
